In [ ]:
import azureml.core
print(azureml.core.VERSION)

In [ ]:
from azureml.core.workspace import Workspace
ws = Workspace.from_config()
ws.get_details()

In [ ]:
%%writefile requirements_interpretability.txt

azureml-core
scikit-learn
azureml-interpret
azureml-dataset-runtime

In [ ]:
from azureml.core import Environment
from azureml.core.environment import CondaDependencies
myenv = Environment.from_pip_requirements(name="scikit_interpretability", file_path="requirements_interpretability.txt")
myenv.register(workspace=ws)

In [ ]:
%%writefile train_interpretability.py

import joblib

from sklearn.datasets import load_diabetes
from sklearn.linear_model import Ridge
import sklearn

from azureml.core import Dataset
from azureml.core.run import Run
import numpy as np
import sklearn

from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration
from azureml.interpret import ExplanationClient
from interpret.ext.blackbox import TabularExplainer
from sklearn.model_selection import train_test_split


dataset_x, dataset_y = load_diabetes(return_X_y=True)
x_train, x_test, y_train, y_test = train_test_split(dataset_x, dataset_y)
model = Ridge().fit(x_train, y_train)

joblib.dump(model, 'sklearn_regression_model.pkl')

print('Model trained')


## Store sample input and output datasets into Azure

np.savetxt('features.csv', x_train, delimiter=',')
np.savetxt('labels.csv', y_train, delimiter=',')
run = Run.get_context()
ws = run.experiment.workspace
datastore = ws.get_default_datastore()
datastore.upload_files(files=['./features.csv', './labels.csv'],
                       target_path='sklearn_regression/',
                       overwrite=True)

input_dataset = Dataset.Tabular.from_delimited_files(path=[(datastore, 'sklearn_regression/features.csv')])
output_dataset = Dataset.Tabular.from_delimited_files(path=[(datastore, 'sklearn_regression/labels.csv')])

#Interpret model

client = ExplanationClient.from_run(run)


# explain predictions on your local machine
explainer = TabularExplainer(model, 
                             x_train,
                            features=["AGE", "SEX", "BMI", "BP", "S1", "S2", "S3", "S4", "S5", "S6"])

global_explanation = explainer.explain_global(x_test)
client.upload_model_explanation(global_explanation, comment='global explanation: all features')

## Register model

model = Model.register(workspace=ws,
                       model_name='interpreted_model',                # Name of the registered model in your workspace.
                       model_path='./sklearn_regression_model.pkl',  # Local file to upload and register as a model.
                       model_framework=Model.Framework.SCIKITLEARN,  # Framework used to create the model.
                       model_framework_version=sklearn.__version__,  # Version of scikit-learn used to create the model.
                       sample_input_dataset=input_dataset,
                       sample_output_dataset=output_dataset,
                       resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=0.5),
                       description='Ridge regression model to predict diabetes progression.',
                       tags={'area': 'diabetes', 'type': 'regression'})

print('Name:', model.name)
print('Version:', model.version)

In [ ]:
from azureml.core import Experiment, ScriptRunConfig
experiment = Experiment(workspace=ws, name='day2-experiment-train')
config = ScriptRunConfig(source_directory='.',
                             script='train_interpretability.py',
                             compute_target='cpu-cluster',
                             environment=myenv)
run = experiment.submit(config)

In [ ]:
aml_url = run.get_portal_url()
print(aml_url)